# Run circuits in Qiskit 1.0

Source: https://github.com/qiskit-community/ibm-quantum-challenge-2024

In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

## IBM Runtime
Real and fake circuits.

### EstimatorV2

In [2]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

obs = ['IZ', 'IX', 'ZI', 'XI', 'ZZ', 'XX']
observables = []
for o in obs:
    observables.append(SparsePauliOp(o))

estimator = Estimator(mode=AerSimulator())
pub = (qc, observables)

job = estimator.run(pubs=[pub])
result = job.result()[0].data.evs
print(result)

[0.02099609 0.00341797 0.02099609 0.00341797 1.         1.        ]


### SamplerV2

In [3]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit import transpile
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke, FakePrague
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()
#pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
#isa_circuit = pm.run(qc)
transp = transpile(qc, backend=FakePrague())
aer_sim = AerSimulator()
sampler = Sampler(mode=FakePrague())
result = sampler.run([transp]).result()
samp_dist = result[0].data.meas.get_counts()
print(samp_dist)

{'11': 508, '00': 492, '01': 14, '10': 10}


In [5]:
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_aer import AerSimulator

### Runtime Service

```python
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions

# API Token
%set_env QXToken=

service = QiskitRuntimeService()
backend = service.backend("ibm_sherbrooke")
estimator = Estimator(backend=backend, options=options)
```

## Qiskit Primitives

In [6]:
from qiskit.primitives import StatevectorSampler

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

sampler = StatevectorSampler()
pub = qc
job_sampler = sampler.run([pub], shots=256)

result_sampler = job_sampler.result()
counts_sampler = result_sampler[0].data.meas.get_counts()
print(counts_sampler)

{'11': 135, '00': 121}


## Pass manager
Useful if you want to work directly with a preset pass manager, `generate_preset_pass_maganer()` is called inside `transpile()`.

In [7]:
from qiskit.primitives import StatevectorEstimator

from qiskit.transpiler import StagedPassManager, InstructionProperties, PassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.plugin import list_stage_plugins
from qiskit.transpiler.timing_constraints import TimingConstraints
from qiskit.transpiler.passes.scheduling import ASAPScheduleAnalysis,PadDynamicalDecoupling

In [8]:
# Define the estimator and pass manager
estimator = StatevectorEstimator()
pm = generate_preset_pass_manager(backend=AerSimulator(), optimization_level=3, seed_transpiler=0) 

classifier = qc.compose(ansatz) 
transpiled_classifier = pm.run(classifier)
transpiled_obs = obs.apply_layout(layout=transpiled_classifier.layout)

pub = (transpiled_classifier, transpiled_obs, params)
job = estimator.run([pub])
result = job.result()[0].data.evs

NameError: name 'ansatz' is not defined